# Database Integration for Clustering Model
### Purpose
This notebook integrates the clustering model with a MySQL database:
- Connects to the database to fetch user preferences
- Processes preferences and predicts the cluster for each user
- Updates the user's assigned cluster in the database
- Ensures error handling and logging for database operations


In [3]:
import joblib
import numpy as np
import mysql.connector
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans

# Load the pre-trained K-means clustering model and MultiLabelBinarizer
kmeans = joblib.load('kmeans_color_model_v11.pkl')
mlb = joblib.load('mlb_encoder_v11.pkl')

# Database connection details
DB_HOST = "54.221.51.240"
DB_USER = "theuser"
DB_PASSWORD = "thepassword"
DB_NAME = "nexa"

# Connect to the database
def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        print("Connected to the database successfully.")
        return connection
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Fetch user preferences from the database
def fetch_user_preferences(email, connection):
    query = "SELECT * FROM client_preferences WHERE email = %s"
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query, (email,))
    user_data = cursor.fetchone()
    if user_data:
        print("User preferences fetched successfully.")
    else:
        print("No data found for the provided email.")
    return user_data

# Process and encode user preferences
def process_user_preferences(user_data):
    user_preferences = {
        "selected_colors": user_data.get("favorite_colors", "").split(", "),
    }
    # Encode user color preferences
    user_colors_encoded = mlb.transform([user_preferences["selected_colors"]])[0]
    return user_colors_encoded

# Update the cluster in the database
def update_user_cluster(email, user_cluster, connection):
    query = "UPDATE client SET user_group = %s WHERE email = %s"
    cursor = connection.cursor()
    cursor.execute(query, (str(user_cluster), email))
    connection.commit()
    print("Cluster group updated successfully in the database.")

# Main function
def main():
    # Example email input
    email = "isuru@gmail.com"
    
    # Step 1: Connect to the database
    connection = connect_to_database()
    if not connection:
        return
    
    # Step 2: Fetch user preferences
    user_data = fetch_user_preferences(email, connection)
    if not user_data:
        connection.close()
        return
    
    # Step 3: Process and encode user preferences
    user_features = process_user_preferences(user_data)
    
    # Step 4: Predict the cluster
    user_cluster = kmeans.predict([user_features])[0]
    print(f"Predicted Cluster for User: Cluster {user_cluster}")
    
    # Step 5: Update the database
    update_user_cluster(email, user_cluster, connection)
    
    # Close the database connection
    connection.close()

# Run the main function
main()


Connected to the database successfully.
User preferences fetched successfully.
Predicted Cluster for User: Cluster 3


C:\Users\User\anaconda3\envs\deep_learning\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  "X does not have valid feature names, but"


Cluster group updated successfully in the database.
